In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
import os
import gzip
import torch
import gzip
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from tabulate import tabulate

Define The class and experiments and load the data.

In [ ]:

# Define a flexible MLP model with a fixed first layer and variable hidden layers
class FlexibleMLP(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers):
        """
        Args:
            input_size (int): Number of input features.
            output_size (int): Number of output classes.
            hidden_layers (list): List of integers specifying the size of each hidden layer after the fixed first layer.
        """
        super(FlexibleMLP, self).__init__()
        layers = []
        
        # Add the fixed first layer
        layers.append(nn.Linear(input_size, 512))
        layers.append(nn.ReLU())
        
        # Add the variable hidden layers
        in_features = 512
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(in_features, hidden_size))
            layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.ReLU())
            in_features = hidden_size

        # Add the output layer
        layers.append(nn.Linear(in_features, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Load the training and test data
train_data = torch.load('../Data/ProcessedData/train_data.pth')
train_features = train_data['features']
train_labels = train_data['labels']

test_data = torch.load('../Data/ProcessedData/test_data.pth')
test_features = test_data['features']
test_labels = test_data['labels']

# Experiment configurations: Various layer setups after the fixed first layer
experiment_configurations = [
    [],                             # Remove hidden layers after the first
    [512],                          # Inital structure
    [512, 256],                     # One small additional hidden layer
    [512, 1024],                    # One large  additional hidden layer
    [512, 256, 128],                # Two small additional layers
    [512, 2048, 1024],              # Two large additional layers
    [512, 256, 128, 64],            # Three small additional layers 
    [512, 4096, 2048, 1024],        # Three large additional layers
    ]      
results = []

Train and save the weights

In [ ]:
# Ensure the Weights directory exists
weights_dir = "Weights"
os.makedirs(weights_dir, exist_ok=True)

# Loop through each architecture configuration for experimentation
for idx, hidden_layers in enumerate(experiment_configurations):
    print(f"\nExperiment {idx+1}: Hidden Layers - [{','.join(map(str, hidden_layers))}]")
    mlp_model = FlexibleMLP(input_size=50, output_size=10, hidden_layers=hidden_layers)
    
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(mlp_model.parameters(), lr=0.01, momentum=0.9)

    # Training loop
    num_epochs = 50
    for epoch in range(num_epochs):
        mlp_model.train()
        optimizer.zero_grad()
        
        # Forward pass
        outputs = mlp_model(train_features)
        
        # Compute loss
        loss = criterion(outputs, train_labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Print loss for the final epoch
        if epoch == num_epochs - 1:
            print(f"Final Epoch Loss: {loss.item():.4f}")

    # Save the model weights in a compressed .gz file
    model_name = f"MLP_HiddenLayers_{'_'.join(map(str, hidden_layers))}_weights.pth.gz"
    model_save_path = os.path.join(weights_dir, model_name)
    with gzip.open(model_save_path, "wb") as f:
        torch.save(mlp_model.state_dict(), f)
    print(f"Compressed model weights for Experiment {idx+1} saved to {model_save_path}")

Load the weights and evaluate each model.


P.S In testing flow after imports, please run the cell of dataloading and class FlexibleMLP (2nd cell) before Prdiction.

In [ ]:
# Ensure the Weights directory exists
weights_dir = "Weights"

# Loop through each architecture configuration for testing
resultsfinal = []
for idx, hidden_layers in enumerate(experiment_configurations):
    print(f"\nTesting Experiment {idx+1}: Hidden Layers - [{', '.join(map(str, hidden_layers))}]")
    mlp_model = FlexibleMLP(input_size=50, output_size=10, hidden_layers=hidden_layers)
    
    # Compressed model file path
    model_name = f"MLP_HiddenLayers_{'_'.join(map(str, hidden_layers))}_weights.pth.gz"
    model_save_path = os.path.join(weights_dir, model_name)

    # Load the model weights from compressed file
    if os.path.exists(model_save_path):
        with gzip.open(model_save_path, "rb") as f:
            mlp_model.load_state_dict(torch.load(f))
        print(f"Model weights for Experiment {idx+1} loaded from compressed file: {model_save_path}")
    else:
        print(f"Model weights for Experiment {idx+1} not found. Skipping this configuration.")
        continue

    # Evaluate the model
    mlp_model.eval()
    with torch.no_grad():
        test_outputs = mlp_model(test_features)
        _, test_predictions = torch.max(test_outputs, 1)

    # Evaluate metrics
    accuracy = accuracy_score(test_labels.numpy(), test_predictions.numpy())
    precision = precision_score(test_labels.numpy(), test_predictions.numpy(), average=None)
    recall = recall_score(test_labels.numpy(), test_predictions.numpy(), average=None)
    f1 = f1_score(test_labels.numpy(), test_predictions.numpy(), average=None)

    # Record results
    resultsfinal.append({
        "Experiment": f"Hidden Layers: [{', '.join(map(str, hidden_layers))}]",
        "Accuracy": accuracy,
        "Precision (Avg)": np.mean(precision),
        "Recall (Avg)": np.mean(recall),
        "F1-Score (Avg)": np.mean(f1),
    })

    # Plot the confusion matrix for the experiment
    cm = confusion_matrix(test_labels.numpy(), test_predictions.numpy())
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=range(10), yticklabels=range(10))
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title(f"Confusion Matrix for Experiment: Hidden Layers - [{', '.join(map(str, hidden_layers))}]")
    plt.show()

# Compile results into a DataFrame
if resultsfinal:  # Check if resultsfinal is not empty
    print(f"Results collected: {len(resultsfinal)} experiments")
    # Print example for debugging
    print("Example Result Entry:", resultsfinal[0])

    df_results = pd.DataFrame(resultsfinal)

    # Ensure the correct columns exist in the DataFrame
    columns = ["Experiment", "Accuracy", "Precision (Avg)", "Recall (Avg)", "F1-Score (Avg)"]
    missing_columns = [col for col in columns if col not in df_results.columns]
    if missing_columns:
        print(f"Warning: Missing columns in results: {missing_columns}")
    else:
        df_results = df_results[columns]

    # Format and display the results summary
    formatted_table = tabulate(df_results, headers="keys", tablefmt="pretty", showindex=False)
    print("Experiment Results Summary:")
    print(formatted_table)
else:
    print("No results found in resultsfinal. Check training or evaluation steps.")